<a href="https://colab.research.google.com/github/bmsjale93/teoria-de-juegos-resuelto/blob/main/Teoria_de_Juegos_e_Inteligencia_Artificial_Alejandro_Delgado_53344428E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nashpy

In [ ]:
import nashpy as nash
import numpy as np

Definimos las matrices de recompensas:

In [ ]:
A = np.array([[-8, 0], [-10, -1]]) # A es el jugador filas.
B = np.array([[-8, -10], [0, -1]]) # B es el jugador columnas.
juego1 = nash.Game(A,B)
juego1

Bi matrix game with payoff matrices:

Row player:
[[ -8   0]
 [-10  -1]]

Column player:
[[ -8 -10]
 [  0  -1]]

Encontramos las estrategias de Equilibrio:

In [ ]:
equilibrios = juego1.support_enumeration()
for eq in equilibrios:
    print(eq)

(array([1., 0.]), array([1., 0.]))


De manera similar, para un juego mas complejo con estrategias dominantes:

In [ ]:
A2 = np.array([[6, 4, 4, 3],
               [5, 6, 0, 5],
               [5, 3, 6, 4],
               [2, 2, 3, 6]]) # A es el jugador filas.

B2 = np.array([[3, 4, 1, 0],
               [4, 3, 2, 1],
               [0, 2, 1, 4],
               [0, 3, 3, 1]]) # B es el jugador columnas.
juego2 = nash.Game(A2, B2)
juego2

Bi matrix game with payoff matrices:

Row player:
[[6 4 4 3]
 [5 6 0 5]
 [5 3 6 4]
 [2 2 3 6]]

Column player:
[[3 4 1 0]
 [4 3 2 1]
 [0 2 1 4]
 [0 3 3 1]]

Las estrategias resultantes son aquellas que no son dominadas:

In [ ]:
equilibrios = juego2.support_enumeration()
for eq in equilibrios:
    print(eq)

(array([0.5, 0.5, 0. , 0. ]), array([0.66666667, 0.33333333, 0.        , 0.        ]))


# Minimax para 3 en Raya

In [ ]:
import time

class Juego:
    def __init__(self):
        self.iniciar_juego()

    def iniciar_juego(self):
        self.estado_actual = [['.','.','.'],
                              ['.','.','.'],
                              ['.','.','.']]

        # X siempre juega primero:
        self.turno_jugador = 'X'

    def dibujar_tablero(self):
        for i in range(0, 3):
            for j in range(0, 3):
                print('{}|'.format(self.estado_actual[i][j]), end=" ")
            print()
        print()

    def es_valido(self, px, py):
      if px < 0 or px > 2 or py < 0 or py > 2:
          return False
      elif self.estado_actual[px][py] != '.':
          return False
      else:
          return True

    def es_fin(self):
      # Victoria en vertical:
      for i in range(0, 3):
          if (self.estado_actual[0][i] != '.' and
              self.estado_actual[0][i] == self.estado_actual[1][i] and
              self.estado_actual[1][i] == self.estado_actual[2][i]):
              return self.estado_actual[0][i]

      # Victoria en Horizontal:
      for i in range(0, 3):
          if (self.estado_actual[i] == ['X', 'X', 'X']):
              return 'X'
          elif (self.estado_actual[i] == ['O', 'O', 'O']):
              return 'O'

      # Victoria diagonal principal:
      if (self.estado_actual[0][0] != '.' and
          self.estado_actual[0][0] == self.estado_actual[1][1] and
          self.estado_actual[0][0] == self.estado_actual[2][2]):
          return self.estado_actual[0][0]

      # Victoria diagonal secundaria:
      if (self.estado_actual[0][2] != '.' and
          self.estado_actual[0][2] == self.estado_actual[1][1] and
          self.estado_actual[0][2] == self.estado_actual[2][0]):
          return self.estado_actual[0][2]

      # Mesa llena:
      for i in range(0, 3):
          for j in range(0, 3):
              # Si hay espacio, seguimos jugando:
              if (self.estado_actual[i][j] == '.'):
                  return None

      # Empate, mesa llena sin victoria:
      return '.'

    # 'O' es max, en este caso la IA:
    def max(self):

        # Posibles valores para max:
        # -1 - pierde
        #  0 - empata
        #  1 - gana

        # Inicializamos el resultado a -2, peor que lo peor
        maxv = -2

        px = None
        py = None

        resultado = self.es_fin()

        # Si el juego acaba, debemos devolver el resultado:
        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif resultado == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    # Esto es una rama del juego:
                    self.estado_actual[i][j] = 'O'
                    (m, min_i, min_j) = self.min()
                    # Fijamos maxv si es necesario:
                    if m > maxv:
                        maxv = m
                        px = i
                        py = j
                    # Vaciamos el tablero
                    self.estado_actual[i][j] = '.'
        return (maxv, px, py)

    # El jugador 'X' es min, un humano en este caso:
    def min(self):

        #Inicio igual que para max:
        minv = 2

        qx = None
        qy = None

        resultado = self.es_fin()

        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif resultado == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    self.estado_actual[i][j] = 'X'
                    (m, max_i, max_j) = self.max()
                    if m < minv:
                        minv = m
                        qx = i
                        qy = j
                    self.estado_actual[i][j] = '.'

        return (minv, qx, qy)

    def jugar(self):
      while True:
          self.dibujar_tablero()
          self.resultado = self.es_fin()

          # Mostrar que el juego termino:
          if self.resultado != None:
              if self.resultado == 'X':
                  print('El ganador es X!')
              elif self.resultado == 'O':
                  print('El ganador es O!')
              elif self.resultado == '.':
                  print("Empate!")

              self.iniciar_juego()
              return

          # If it's player's turn
          if self.turno_jugador == 'X':

              while True:

                  inicio = time.time()
                  (m, qx, qy) = self.min()
                  fin = time.time()
                  print('Evaluacion: {}s'.format(round(fin - inicio, 7)))
                  print('Movimientos Recomendados: X = {}, Y = {}'.format(qx, qy))

                  px = int(input('X coordenada: '))
                  py = int(input('Y coordenada: '))

                  (qx, qy) = (px, py)

                  if self.es_valido(px, py):
                      self.estado_actual[px][py] = 'X'
                      self.turno_jugador = 'O'
                      break
                  else:
                      print('Movimiento no valido, prueba otra vez.')

          # Turno de la IA
          else:
              (m, px, py) = self.max()
              self.estado_actual[px][py] = 'O'
              self.turno_jugador = 'X'

In [ ]:
g = Juego()
g.jugar()

.| .| .| 
.| .| .| 
.| .| .| 

Evaluacion: 4.118149s
Movimientos Recomendados: X = 0, Y = 0
X coordenada: 2
Y coordenada: 0
.| .| .| 
.| .| .| 
X| .| .| 

.| .| .| 
.| O| .| 
X| .| .| 

Evaluacion: 0.0305676s
Movimientos Recomendados: X = 0, Y = 0
X coordenada: 1
Y coordenada: 3
Movimiento no valido, prueba otra vez.
Evaluacion: 0.0593412s
Movimientos Recomendados: X = 0, Y = 0
X coordenada: 1
Y coordenada: 2
.| .| .| 
.| O| X| 
X| .| .| 

.| O| .| 
.| O| X| 
X| .| .| 

Evaluacion: 0.0006654s
Movimientos Recomendados: X = 2, Y = 1
X coordenada: 2
Y coordenada: 1
.| O| .| 
.| O| X| 
X| X| .| 

.| O| .| 
.| O| X| 
X| X| O| 

Evaluacion: 9.94e-05s
Movimientos Recomendados: X = 0, Y = 0
X coordenada: 0
Y coordenada: 0
X| O| .| 
.| O| X| 
X| X| O| 

X| O| .| 
O| O| X| 
X| X| O| 

Evaluacion: 2.22e-05s
Movimientos Recomendados: X = 0, Y = 2
X coordenada: 0
Y coordenada: 2
X| O| X| 
O| O| X| 
X| X| O| 

Empate!


# Ejercicio: ¿Puedes modificar las funciones min y max para podar las ramas innecesarias? ¿Que le pasa al tiempo de procesamiento de la busqueda de la IA?

Modifica las funciones min, max y jugar con los ejemplos siguientes rellenando los espacios:

In [ ]:
def max_alfa_beta(self, alfa, beta):
        maxv = -2
        px = None
        py = None

        resultado = self.es_final()

        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif result == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    self.estado_actual[i][j] = 'O'
                    (m, min_i, in_j) = self.min_alfa_beta(alfa, beta)
                    if m > maxv:
                        maxv = m
                        px = i
                        py = j
                    self.estado_actual[i][j] = '.'


                    if maxv >= beta:
                        return (maxv, px, py)

                    if maxv > alfa:
                        alfa = maxv

        return (maxv, px, py)

In [ ]:
def min_alfa_beta(self, alfa, beta):

        minv = 2

        qx = None
        qy = None

        resultado = self.es_final()

        if resultado == 'X':
            return (-1, 0, 0)
        elif resultado == 'O':
            return (1, 0, 0)
        elif resultado == '.':
            return (0, 0, 0)

        for i in range(0, 3):
            for j in range(0, 3):
                if self.estado_actual[i][j] == '.':
                    self.estado_actual[i][j] = 'X'
                    (m, max_i, max_j) = self.max_alfa_beta(alfa, beta)
                    if m < minv:
                        minv = m
                        qx = i
                        qy = j
                    self.estado_actual[i][j] = '.'

                    if minv <= alfa:
                        return (minv, qx, qy)

                    if minv < beta:
                        beta = minv

        return (minv, qx, qy)

In [ ]:
def jugar_alfa_beta(self):
  while True:
    self.dibujar_tablero()
    self.resultado = self.es_final()

    if self.resultado != None:

      if self.resultado == 'X':
        print('Gana X!')

      elif self.resultado == 'O':
        print('Gana O!')

      elif self.resultado == '.':
        print("Empate")

      self.inicializar_estado()
      return

      if self.turno == 'X':
        while True:
          inicio = time.time()
          (m, qx, qy) = self.min_alfa_beta(-2, 2)
          fin = time.time()
          print('Tiempo de evaluacion: {}s'.format(round(fin - inicio, 7)))
          print('Movimiento recomendado: X = {}, Y = {}'.format(qx, qy))

          px = int(input('X coordenada: '))
          py = int(input('Y coordenada: '))

          # No son necesarias, estas variables se utilizan
          # para mostrar al jugador el movimiento sugerido
          # por la IA, no hay que asignarle valores.
          # qx = __
          # qy = __

      if self.es_movimiento_valido(px, py):
          self.estado_actual[px][py] = 'X'
          self.turno = 'O'
          break
      else:
          print('Movimiento no válido. Prueba con otro movimiento.')

  else:
         (m, px, py) = self.max_alfa_beta(-2, 2)
         self.estado_actual[px][py] = 'O'
         self.turno = 'X'